# How do I read this data?

There's a variety of samples script out there for using the HRRR Zarr data. They don't all load the data the same way, and how to load it can be pretty confusing. This guide provides our current (as of writing) recommendations for the "best" ways to access the data, with weight given to performance (speed), code length, and a preference for high-level APIs. In the examples, as much as possible is written as a function you should just be able to copy and paste.

There are two main paths for reading the data:
* xarray.open_mfdataset  
* HRRR_chunk_index.h5

The first gives you the whole grid with a lot of metadata, the second is faster if you want data from many model runs and only want a single grid point or region.

The open_mfdataset method is somewhat complex since the projection and other metadata need extra setup steps. The chunk index method is complex since no API currently supports reading zarr data from a single chunk rather than the whole group or array.

## For Surface Analysis Files

### Create the https url for direct download

In [9]:
import datetime

def create_analysis_chunk_url(level, param_short_name, hour, chunk_id):
    url = hour.strftime("https://hrrrzarr.s3.amazonaws.com/sfc/%Y%m%d/%Y%m%d_%Hz_anl.zarr/")
    url += f"{level}/{param_short_name}/{level}/{param_short_name}/{chunk_id}"
    return url

create_analysis_chunk_url("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7), "4.3")

'https://hrrrzarr.s3.amazonaws.com/sfc/20210101/20210101_07z_anl.zarr/1000mb/TMP/1000mb/TMP/4.3'

### Create the S3 url

In [41]:
import datetime

def create_analysis_s3_url(level, param_short_name, hour, chunk_id, prefix=False):
    url = "s3://hrrrzarr/" if prefix else "" # If we use boto3 we only want the path starting at the bucket
    url += hour.strftime("sfc/%Y%m%d/%Y%m%d_%Hz_anl.zarr/")
    url += f"{level}/{param_short_name}/{level}/{param_short_name}/{chunk_id}"
    return url

print(create_analysis_s3_url("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7), "4.3", prefix=True))

s3://hrrrzarr/sfc/20210101/20210101_07z_anl.zarr/1000mb/TMP/1000mb/TMP/4.3


In [14]:
# These are what you need to load the whole grid into xarray using open_mfdataset
def create_analysis_group_and_subgroup_urls(level, param_short_name, hour):
    root_url = hour.strftime("s3://hrrrzarr/sfc/%Y%m%d/%Y%m%d_%Hz_anl.zarr/")
    group_url = root_url + f"{level}/{param_short_name}"
    subgroup_url = group_url + f"/{level}"
    return [group_url, subgroup_url]

print(create_analysis_group_and_subgroup_urls("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7)))

['s3://hrrrzarr/sfc/20210101/20210101_07z_anl.zarr/1000mb/TMP', 's3://hrrrzarr/sfc/20210101/20210101_07z_anl.zarr/1000mb/TMP/1000mb']


### Open the (whole grid) zarr array

We recommend using this method if you're interested in the whole or most of the grid, or if you're only downloading one model run. It allows you to leverage high-level APIs and metadata, unlike lookup by chunk.

In [23]:
import s3fs
import xarray as xr
import metpy

def load_dataset(urls):
    fs = s3fs.S3FileSystem(anon=True)
    
    # open_mfdataset is required because the zarr data is slightly malformed:
    # the main data variable has a / in its name (e.g. "1000mb/TMP") that becomes a nested subgroup
    ds = xr.open_mfdataset([s3fs.S3Map(url, s3=fs) for url in urls], engine='zarr')
    
    # add the projection data
    ds = ds.rename(projection_x_coordinate="x", projection_y_coordinate="y")
    ds = ds.metpy.assign_crs(grid_mapping_name="lambert_conformal_conic", longitude_of_central_meridian=-97.5,
                                 latitude_of_projection_origin=38.5,
                                 standard_parallel=38.5)
    ds = ds.metpy.assign_latitude_longitude()
    
    # upgrade time to a coordinate - necessary to later combine data from multiple hours
    ds = ds.set_coords("time")
    
    return ds

load_dataset(create_analysis_group_and_subgroup_urls("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7)))

<xarray.Dataset>
Dimensions:                  (x: 1799, y: 1059)
Coordinates:
  * x                        (x) float64 -2.698e+06 -2.695e+06 ... 2.696e+06
  * y                        (y) float64 -1.587e+06 -1.584e+06 ... 1.587e+06
    time                     datetime64[ns] ...
    metpy_crs                object Projection: lambert_conformal_conic
    latitude                 (y, x) float64 21.09 21.1 21.1 ... 47.87 47.86
    longitude                (y, x) float64 -122.7 -122.6 ... -61.05 -61.01
Data variables:
    TMP                      (y, x) float16 dask.array<chunksize=(150, 150), meta=np.ndarray>
    forecast_period          timedelta64[ns] ...
    forecast_reference_time  datetime64[ns] ...
    height                   float64 ...
    pressure                 float64 ...

**Troubleshoot:** In my Jupyter notebook setup, the pyproj package gives the following error when it tries looking up the projection info:

```
CRSError: Invalid datum string: urn:ogc:def:datum:EPSG::6326: (Internal Proj Error: proj_create: SQLite error on SELECT name, ellipsoid_auth_name, ellipsoid_code, prime_meridian_auth_name, prime_meridian_code, publication_date, frame_reference_epoch, deprecated FROM geodetic_datum WHERE auth_name = ? AND code = ?: no such column: publication_date)
```

I believe this is because I run Jupyter from a conda environment that's different than the kernel Jupyter is using. In any case, there's an easy fix:

In [22]:
import pyproj
pyproj.datadir.set_data_dir("/Users/<user>/.conda/envs/<conda_env>/share/proj")

If the metpy way of handling the projection still doesn't work, you can also just work with the projection from cartopy when you need it:

In [20]:
import cartopy.crs as ccrs

projection = ccrs.LambertConformal(central_longitude=-97.5,
                             central_latitude=38.5,
                             standard_parallels=[38.5])

### Combine different hours (whole grid)

In [25]:
# Look at the 0700z run for both the 1st and 2nd of January
ds1 = load_dataset(create_analysis_group_and_subgroup_urls("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7)))
ds2 = load_dataset(create_analysis_group_and_subgroup_urls("1000mb", "TMP", datetime.datetime(2021, 1, 2, 7)))

xr.concat([ds1, ds2], dim="time")

<xarray.Dataset>
Dimensions:                  (time: 2, x: 1799, y: 1059)
Coordinates:
  * x                        (x) float64 -2.698e+06 -2.695e+06 ... 2.696e+06
  * y                        (y) float64 -1.587e+06 -1.584e+06 ... 1.587e+06
  * time                     (time) datetime64[ns] 2021-01-01T07:00:00 2021-0...
    metpy_crs                object Projection: lambert_conformal_conic
    latitude                 (y, x) float64 21.09 21.1 21.1 ... 47.87 47.86
    longitude                (y, x) float64 -122.7 -122.6 ... -61.05 -61.01
Data variables:
    TMP                      (time, y, x) float16 dask.array<chunksize=(1, 150, 150), meta=np.ndarray>
    forecast_period          (time) timedelta64[ns] 00:00:00 00:00:00
    forecast_reference_time  (time) datetime64[ns] 2021-01-01T07:00:00 2021-0...
    height                   (time) float64 1e+03 1e+03
    pressure                 (time) float64 2.5e+04 2.5e+04

### Download data by chunk - one grid point, nearest latitude and longitude

If you're downloading a lot of hours of data and don't need the whole grid, the xarray method will be extremely slow. Instead, the data is chunked based on geospatial location, so you can download only the necessary chunks of the zarr array. Doing this is lower-level and won't decompress or read metadata automatically.

First, we get the chunk index file:

In [26]:
! wget "https://hrrrzarr.s3.amazonaws.com/grid/HRRR_chunk_index.h5"

--2021-06-18 12:45:48--  https://hrrrzarr.s3.amazonaws.com/grid/HRRR_chunk_index.h5
Resolviendo hrrrzarr.s3.amazonaws.com (hrrrzarr.s3.amazonaws.com)... 52.219.121.18
Conectando con hrrrzarr.s3.amazonaws.com (hrrrzarr.s3.amazonaws.com)[52.219.121.18]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 38160716 (36M) [application/x-www-form-urlencoded]
Grabando a: «HRRR_chunk_index.h5.1»

HRRR_chunk_index.h5 100%[===================>]  36.39M  29.8MB/s    en 1.2s    

2021-06-18 12:45:50 (29.8 MB/s) - «HRRR_chunk_index.h5.1» guardado [38160716/38160716]



In [28]:
chunk_index = xr.open_dataset("HRRR_chunk_index.h5", engine="scipy") # Currently only scipy works with this file!
chunk_index

<xarray.Dataset>
Dimensions:     (x: 1799, y: 1059)
Coordinates:
  * x           (x) float64 -2.701e+06 -2.698e+06 ... 2.697e+06 2.7e+06
  * y           (y) float64 -1.581e+06 -1.578e+06 ... 1.584e+06 1.587e+06
Data variables:
    chunk_id    (x, y) object ...
    index_x     (x) int32 ...
    chunk_x     (x) int32 ...
    in_chunk_x  (x) int32 ...
    latitude    (y, x) float64 ...
    longitude   (y, x) float64 ...
    index_y     (y) int32 ...
    chunk_y     (y) int32 ...
    in_chunk_y  (y) int32 ...

Next, we need to get the nearest point:

In [38]:
import cartopy.crs as ccrs

projection = ccrs.LambertConformal(central_longitude=-97.5,
                             central_latitude=38.5,
                             standard_parallels=[38.5])

def get_nearest_point(projection, chunk_index, latitude, longitude):
    x, y = projection.transform_point(-111.8910, 40.7608, ccrs.PlateCarree())
    return chunk_index.sel(x=x, y=y, method="nearest")

nearest_point = get_nearest_point(projection, chunk_index, -111.8910, 40.7608)
chunk_id = nearest_point.chunk_id.values
print(chunk_id)

4.3


Then, we retrieve the chunk. We can't use xarray, since it doesn't understand how to load a single chunk of a zarr array. We could simply retrieve the data from the https url, but if we're going to do this request more than once (e.g. getting the same chunk for every hour of the day), the fastest way is to use boto3. 

In [42]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config

# Don't recreate this resource in a loop! That caused a 3-4x slowdown for me.
s3 = boto3.resource(service_name='s3', region_name='us-west-1', config=Config(signature_version=UNSIGNED))

def retrieve_object(s3, s3_url):
    obj = s3.Object('hrrrzarr', s3_url)
    return obj.get()['Body'].read()

url = create_analysis_s3_url("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7), chunk_id)
print(url)
compressed_data = retrieve_object(s3, url)

sfc/20210101/20210101_07z_anl.zarr/1000mb/TMP/1000mb/TMP/4.3


In [45]:
import numcodecs as ncd
import numpy as np

# Surface-level pressure requires dtype="<f4", the default should be correct for everything else
def decompress_analysis_chunk(data, dtype="<f2"):
    buffer = ncd.blosc.decompress(data)
    chunk = np.frombuffer(buffer, dtype=dtype)
    data_array = np.reshape(chunk, (150, 150))
    return data_array

chunk_data = decompress_analysis_chunk(compressed_data)

Once we have the decompressed data for the whole chunk, we need to index into the chunk to get the value at our point:

In [46]:
chunk_data[nearest_point.in_chunk_x.values, nearest_point.in_chunk_y.values]

279.5

### Download data by chunk - latitude/longitude region

Here we get a subportion of the grid based on latitude/longitude bounds.

In [51]:
lat_top = 39
lat_bottom = 34
lon_top = -107
lon_bottom = -110 # Four Corners region

def check_boundaries(data):
    return (lat_bottom < data.latitude) & (data.latitude < lat_top) & (
        lon_bottom < data.longitude) & (data.longitude < lon_top)

area = chunk_index.where(check_boundaries, drop=True)
area

<xarray.Dataset>
Dimensions:     (x: 110, y: 194)
Coordinates:
  * x           (x) float64 -1.151e+06 -1.148e+06 ... -8.268e+05 -8.238e+05
  * y           (y) float64 -4.504e+05 -4.474e+05 ... 1.243e+05 1.273e+05
Data variables:
    chunk_id    (x, y) object nan nan nan nan nan nan ... nan nan nan nan nan
    index_x     (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    chunk_x     (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    in_chunk_x  (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    latitude    (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    longitude   (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    index_y     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    chunk_y     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    in_chunk_y  (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan

Note that while we've dropped rows and columns with no datapoints in the box, we still have lots of null datapoints where either the projection x or the projection y coordinate is sometimes within the box, but the combination is not.

Let's get the unique chunk ids from this dataset.

In [50]:
def get_unique(data):
    # We have to implement our own "unique" logic since missing values are NaN (a float) and the rest are string
    data = data.fillna(None).values.flatten()
    data = data[data != None]
    return np.unique(data)

chunk_ids = get_unique(area.chunk_id)

array(['2.3', '2.4', '3.3', '3.4'], dtype=object)

Now we download the chunks from AWS, decompress, and join the chunk data with the grid data.

In [77]:
def get_chunk(level, param_short_name, hour, chunk_id):
    # retrieve data as before
    url = create_analysis_s3_url(level, param_short_name, hour, chunk_id)
    compressed_data = retrieve_object(s3, url)
    chunk_data = decompress_analysis_chunk(compressed_data)
    
    # combine retrieved data with the chunk grid
    chunk_xarray = ds.where(lambda x: x.chunk_id == chunk_id, drop=True)
    chunk_xarray[param_short_name] = (("y", "x"), chunk_data)
    return chunk_xarray

def get_chunks_combined(level, param_short_name, hour, chunk_ids):
    chunks = [get_chunk(level, param_short_name, hour, chunk_id) for chunk_id in chunk_ids]
    return xr.merge(chunks)

    
data = get_chunks_combined("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7), chunk_ids)
data

<xarray.Dataset>
Dimensions:     (x: 300, y: 300)
Coordinates:
  * x           (x) float64 -1.35e+06 -1.347e+06 ... -4.543e+05 -4.512e+05
  * y           (y) float64 -6.838e+05 -6.808e+05 ... 2.082e+05 2.112e+05
Data variables:
    chunk_id    (x, y) object '2.3' '2.3' '2.3' '2.3' ... '3.4' '3.4' '3.4'
    index_x     (x, y) float64 450.0 450.0 450.0 450.0 ... 749.0 749.0 749.0
    chunk_x     (x, y) float64 3.0 3.0 3.0 3.0 3.0 3.0 ... 4.0 4.0 4.0 4.0 4.0
    in_chunk_x  (x, y) float64 0.0 0.0 0.0 0.0 0.0 ... 149.0 149.0 149.0 149.0
    latitude    (y, x) float64 31.4 31.41 31.41 31.42 ... 40.27 40.27 40.27
    longitude   (y, x) float64 -111.6 -111.6 -111.6 ... -102.9 -102.8 -102.8
    index_y     (y, x) float64 300.0 300.0 300.0 300.0 ... 599.0 599.0 599.0
    chunk_y     (y, x) float64 2.0 2.0 2.0 2.0 2.0 2.0 ... 3.0 3.0 3.0 3.0 3.0
    in_chunk_y  (y, x) float64 0.0 0.0 0.0 0.0 0.0 ... 149.0 149.0 149.0 149.0
    TMP         (y, x) float16 287.0 287.0 287.0 287.0 ... 280.0 280.0 279.8

This has all the data for the four chunks. If we truly only want to look at the area of interest, we need to apply the filter again.

In [55]:
data.where(check_boundaries, drop=True)

<xarray.Dataset>
Dimensions:     (x: 110, y: 194)
Coordinates:
  * x           (x) float64 -1.151e+06 -1.148e+06 ... -8.268e+05 -8.238e+05
  * y           (y) float64 -4.504e+05 -4.474e+05 ... 1.243e+05 1.273e+05
Data variables:
    chunk_id    (x, y) object nan nan nan nan nan nan ... nan nan nan nan nan
    index_x     (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    chunk_x     (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    in_chunk_x  (x, y) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    latitude    (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    longitude   (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    index_y     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    chunk_y     (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    in_chunk_y  (y, x) float64 nan nan nan nan nan nan ... nan nan nan nan nan
    TMP         (y, x) float16 nan nan nan nan nan nan ... nan nan nan nan nan

### Combine different hours (partial grid)

In [85]:
def get_chunk_with_time(level, param_short_name, hour, chunk_id):
    chunk = get_chunk(level, param_short_name, hour, chunk_id)
    chunk["time"] = hour
    chunk = chunk.set_coords("time")
    return chunk

def get_multi_time_data(level, param_short_name, hours, chunk_ids):
    single_hour_data = [xr.merge([get_chunk_with_time(level, param_short_name, hour, chunk_id)
                                 for chunk_id in chunk_ids]) for hour in hours]
    return xr.concat(single_hour_data, dim="time")

# Example - getting multiple hours from a start time
start_hour = datetime.datetime(2021, 1, 1, 7)
hours = [start_hour + datetime.timedelta(hours=delta) for delta in range(2)]

get_multi_time_data("1000mb", "TMP", hours, chunk_ids)

<xarray.Dataset>
Dimensions:     (time: 2, x: 300, y: 300)
Coordinates:
  * x           (x) float64 -1.35e+06 -1.347e+06 ... -4.543e+05 -4.512e+05
  * y           (y) float64 -6.838e+05 -6.808e+05 ... 2.082e+05 2.112e+05
  * time        (time) datetime64[ns] 2021-01-01T07:00:00 2021-01-01T08:00:00
Data variables:
    chunk_id    (time, x, y) object '2.3' '2.3' '2.3' ... '3.4' '3.4' '3.4'
    index_x     (time, x, y) float64 450.0 450.0 450.0 ... 749.0 749.0 749.0
    chunk_x     (time, x, y) float64 3.0 3.0 3.0 3.0 3.0 ... 4.0 4.0 4.0 4.0 4.0
    in_chunk_x  (time, x, y) float64 0.0 0.0 0.0 0.0 ... 149.0 149.0 149.0 149.0
    latitude    (time, y, x) float64 31.4 31.41 31.41 ... 40.27 40.27 40.27
    longitude   (time, y, x) float64 -111.6 -111.6 -111.6 ... -102.8 -102.8
    index_y     (time, y, x) float64 300.0 300.0 300.0 ... 599.0 599.0 599.0
    chunk_y     (time, y, x) float64 2.0 2.0 2.0 2.0 2.0 ... 3.0 3.0 3.0 3.0 3.0
    in_chunk_y  (time, y, x) float64 0.0 0.0 0.0 0.0 ... 149.0 149.0 149.0 149.0
    TMP         (time, y, x) float16 287.0 287.0 287.0 ... 279.2 279.2 279.0

## For Surface Forecast Files

I'll go into more detail on this later, but there are (at least) four main differences for accessing surface forecast files:

* fcst instead of anl in the urls
* chunk ids have 0. prepended to them in the urls
* there's already a time dimension (representing forecast time) so if you're combining based on model run time you might need to do that a little differently
* the decompression function needs to be a little different to accommodate the new dimension

In [71]:
import datetime

def create_forecast_s3_url(level, param_short_name, hour, chunk_id, prefix=False):
    url = "s3://hrrrzarr/" if prefix else "" # If we use boto3 we only want the path starting at the bucket
    url += hour.strftime("sfc/%Y%m%d/%Y%m%d_%Hz_fcst.zarr/")
    url += f"{level}/{param_short_name}/{level}/{param_short_name}/0.{chunk_id}"
    return url

print(create_forecast_s3_url("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7), "4.3", prefix=True))

s3://hrrrzarr/sfc/20210101/20210101_07z_fcst.zarr/1000mb/TMP/1000mb/TMP/0.4.3


In [72]:
# These are what you need to load the whole grid into xarray using open_mfdataset
def create_forecast_group_and_subgroup_urls(level, param_short_name, hour):
    root_url = hour.strftime("s3://hrrrzarr/sfc/%Y%m%d/%Y%m%d_%Hz_fcst.zarr/")
    group_url = root_url + f"{level}/{param_short_name}"
    subgroup_url = group_url + f"/{level}"
    return [group_url, subgroup_url]

print(create_forecast_group_and_subgroup_urls("1000mb", "TMP", datetime.datetime(2021, 1, 1, 7)))

['s3://hrrrzarr/sfc/20210101/20210101_07z_fcst.zarr/1000mb/TMP', 's3://hrrrzarr/sfc/20210101/20210101_07z_fcst.zarr/1000mb/TMP/1000mb']


In [73]:
# Surface-level pressure requires dtype="<f4", the default should be correct for everything else
def decompress_forecast_chunk(data, dtype="<f2"):
    buffer = ncd.blosc.decompress(data)
    chunk = np.frombuffer(buffer, dtype=dtype)
    entry_size = 22500
    data_array = np.reshape(chunk, (len(chunk)//entry_size, 150, 150))
    return data_array